In [30]:
import torch;
import torch.nn as nn;
from sklearn.datasets import fetch_olivetti_faces

#将图像转为张量
from torchvision.transforms.v2 import ToTensor; 

#数据捆绑 attach
from torch.utils.data import DataLoader



In [31]:
#定义超参数
LR = 1e-3;

#训练的轮次
epochs = 5;

BATCH_SIZE = 64;


In [32]:
facesData = fetch_olivetti_faces(data_home='./face_data',shuffle=True)

print(facesData)

{'data': array([[0.6694215 , 0.6363636 , 0.6487603 , ..., 0.08677686, 0.08264463,
        0.07438017],
       [0.76859504, 0.75619835, 0.74380165, ..., 0.48347107, 0.6280992 ,
        0.6528926 ],
       [0.37190083, 0.34710744, 0.3677686 , ..., 0.7066116 , 0.6818182 ,
        0.5495868 ],
       ...,
       [0.55785125, 0.60330576, 0.6570248 , ..., 0.17768595, 0.20661157,
        0.19421488],
       [0.5206612 , 0.5206612 , 0.53305787, ..., 0.46694216, 0.43801653,
        0.43801653],
       [0.3966942 , 0.3677686 , 0.3429752 , ..., 0.37190083, 0.26859504,
        0.29752067]], dtype=float32), 'images': array([[[0.6694215 , 0.6363636 , 0.6487603 , ..., 0.14049587,
         0.2603306 , 0.30165288],
        [0.661157  , 0.62396693, 0.6652893 , ..., 0.17355372,
         0.1570248 , 0.29752067],
        [0.6280992 , 0.6570248 , 0.6446281 , ..., 0.22727273,
         0.12809917, 0.22727273],
        ...,
        [0.16115703, 0.16115703, 0.13636364, ..., 0.08677686,
         0.09504132, 0.07

In [33]:
image = torch.tensor(facesData.data)

targets = torch.tensor(facesData.target)


In [34]:

dataSet = [(img,ilb) for img,ilb in zip(image,targets)]
print(dataSet)


[(tensor([0.6694, 0.6364, 0.6488,  ..., 0.0868, 0.0826, 0.0744]), tensor(13)), (tensor([0.7686, 0.7562, 0.7438,  ..., 0.4835, 0.6281, 0.6529]), tensor(30)), (tensor([0.3719, 0.3471, 0.3678,  ..., 0.7066, 0.6818, 0.5496]), tensor(34)), (tensor([0.2810, 0.3595, 0.4050,  ..., 0.5372, 0.2603, 0.1446]), tensor(19)), (tensor([0.3760, 0.4752, 0.5744,  ..., 0.3802, 0.3636, 0.4174]), tensor(24)), (tensor([0.2066, 0.2810, 0.3678,  ..., 0.2521, 0.2438, 0.2727]), tensor(6)), (tensor([0.4339, 0.5702, 0.6405,  ..., 0.2231, 0.2231, 0.2273]), tensor(15)), (tensor([0.3884, 0.4628, 0.5537,  ..., 0.1570, 0.1529, 0.1818]), tensor(26)), (tensor([0.5537, 0.5744, 0.5868,  ..., 0.4132, 0.3636, 0.3554]), tensor(14)), (tensor([0.1529, 0.1736, 0.2686,  ..., 0.3388, 0.2149, 0.1529]), tensor(21)), (tensor([0.1033, 0.2190, 0.1777,  ..., 0.4380, 0.4752, 0.2769]), tensor(3)), (tensor([0.4421, 0.6281, 0.5909,  ..., 0.1198, 0.0785, 0.0868]), tensor(13)), (tensor([0.6116, 0.6612, 0.6860,  ..., 0.0620, 0.0620, 0.0909]), 

In [35]:
from torch.utils.data import DataLoader
dataLoader = DataLoader(dataSet, batch_size=32, shuffle=True)

In [47]:
#使用gpu
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

print(device)

mps


In [52]:
from modelBranch import NNBranch;
model = NNBranch({
    'a': [4096, 2048],
    'b': [2048, 1024],
    'c': [1024, 516],
    'd': [516, 40],
})#.to(device) #GPU加速

In [37]:
lossFn = nn.CrossEntropyLoss();#交叉商计算损失
optimizer = torch.optim.AdamW(model.parameters(),lr = LR, betas=(0.95,0.999)) 

In [53]:
model.train() #正则化 归一化 生效 
for item in range(epochs) :
    for data,target in dataLoader:
        #向前运算
        output = model(data.reshape(-1,4096))
        print(output)
        #计算损失
        loss = lossFn(output,target)
        #反向传播 执行梯度下降 优化神经网络参数 是损失降低 提高模型性能
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()
        # print(model[0].weight.grad)  查看第一层权重的梯度
        optimizer.step()
    print(loss.item())

tensor([[-0.0111,  0.0284,  0.0449,  ...,  0.0458, -0.0352,  0.0026],
        [-0.0146,  0.0224,  0.0507,  ...,  0.0426, -0.0298,  0.0058],
        [-0.0169,  0.0160,  0.0488,  ...,  0.0388, -0.0331, -0.0067],
        ...,
        [-0.0118,  0.0219,  0.0508,  ...,  0.0506, -0.0380, -0.0036],
        [-0.0149,  0.0209,  0.0469,  ...,  0.0541, -0.0496,  0.0078],
        [-0.0085,  0.0263,  0.0447,  ...,  0.0452, -0.0416,  0.0028]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0122,  0.0241,  0.0499,  ...,  0.0466, -0.0323,  0.0032],
        [-0.0054,  0.0231,  0.0491,  ...,  0.0417, -0.0389,  0.0074],
        [-0.0106,  0.0246,  0.0419,  ...,  0.0459, -0.0346,  0.0038],
        ...,
        [-0.0141,  0.0130,  0.0542,  ...,  0.0477, -0.0187,  0.0033],
        [-0.0159,  0.0157,  0.0510,  ...,  0.0483, -0.0361,  0.0008],
        [-0.0138,  0.0102,  0.0520,  ...,  0.0476, -0.0345,  0.0024]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0102,  0.0256,  0.0451,  ...,  0.0506, -0.0401, -0.003